# LingPy tutorial

## What is LingPy?

LingPy is a suite of open-source Python modules for sequence comparison, distance analyses, data operations and visualization methods in quantitative historical linguistics. The main idea of LingPy is to provide a software package which, on the one hand, integrates different methods for data analysis in quantitative historical linguistics within a single framework, and, on the other hand, serves as an interface for the preparation and analysis of linguistic data using biological software packages. [LINGPY] The main idea behind it is to combine different approaches to historical linguistics and evolutionary biology into a framework that closely models the most important aspects of the comparative method, allowing experts to focus in both theoretical and specific historical questions while the method takes care of the repetitive and schematic work.

This tutorial illustrates how to use LingPy for the carrying out automatic tasks on historical linguistics: phonetic alignments from wordlists, cognate and borrowing detection, calculation of lexicostatistic distances and affinities among languages, and construction and visualization of language phylogenies. The instructions assume that both Python and LingPy have been properly installed, and that the user as intermediate or advanced proficiency with Python.

## On historical linguistics

The comparative method (Meiller 1925 [1954], Weiss 2014) has sucessfully elucidated the history of a wide range of language families of varying size and age (Baldi 1990, Campbell & Poser 2008) and external evidence has ofter confirmed the validity of the findings (McMahon & McMahon 2005: 10-14). The comparative method is not just a simple technique, but rather an *overarching framework* to study language history (Fox 1995, Jarceva 1990, Klimov 1990, Ross & Durie 1996), which is supported by an underlying workflow that scholas implicitly follow. The most crucial part is the identification of *cognate words* and regular *sound correspondences*. The *iterative character* of the workflow requires repetition in all steps. Iteration is important to address circularity problems: *cognate words* can, for example, only be identified with the help of regular *sound correspondences*, but sound correspondences themselves occur only in cognate words. An iterative procedure circumvents this problem by starting with an initial hypothesis regarding sound correspondences and cognate words which is the constantly revised. [CALC-PROJECT, 6]

## How to load LingPy

Assuming one has a working Python and LingPy installation, the software can be loaded either in a script file or in the interactive shell using the standard procedures for loading libraries in Python. Here we demonstrate how to load the library; loading it as `from lingpy import *` is not recommended, as it pollutes the namespace and make maintainance harder; if you prefer a less-verbose alternative, we suggest loading it with `import lingpy as lp`. Please remember that the first time you load the library it will probably show a lot of information about parameters and models being loaded and cached; this is intentional and expected. The path to the cached data depends on your system, but in common Linux setups it will probably reside in the `~/.cache/lingpy` directory.

In [1]:
import lingpy
from lingpy.tests.util import test_data # used in this tutorial

# query some properties from the library  confirming that it was properly loaded;
# please remember that your output might be different (for example, in the path
# of installation) and that this is not needed in production code, but might
# help debugging
print('lingpy name: ', lingpy.__name__)
print('lingpy version: ', lingpy.__version__)
print('lingpy author: ', lingpy.__author__)
print('lingpy date: ', lingpy.__date__)
print('lingpy path: ', lingpy.__path__)

lingpy name:  lingpy
lingpy version:  2.5
lingpy author:  Johann-Mattis List, Robert Forkel (with contributions by Steven Moran, Taraka Rama, Johannes Dellert, Frank Nagel, Peter Bouda, and Simon Greenhill)
lingpy date:  2016-21-04
lingpy path:  ['/home/tiago/lingpy/lingpy']


## Your first alignments

Although less common in traditional historical linguistics, phonetic alignment plays a crucial role in automatic approaches, with alignment analyses being currently used in many different subfields, such as dialectology (Prokić et al., 2009), phylogenetic reconstruction (Holman et al., 2011) and cognate detection (List, 2012a). Furthermore, alignment analyses are very useful for data visualization, since they directly show which sound segments correspond in cognate words. [LIST2013]

Before we dive into the normal usage of lingpy, with wordlists, multiple alignments or reconstruction of language phylogenies, it is valid to experiment with basic pairwise aligments to understand what could be considered the core of lingpy. Here we are only going to study how to use lingpy for performing such tasks, for detailed explanation on the inner workings and the theory behind such taks, please refer to the recommended bibliography, such as LIST2014.

Pairwise alignment works on two sequences of phonetic units, which are not necessarily "phonemes" in their IPA representation. In fact, while a model that allows to compare sequences with detailed phonetic or phonological representations is possible, as we will see the ones distributed with lingpy don't compare sounds, but sound classes. The advantages and disadvantages of such approach are discussed in the suggest bibliography.

Here we will run some tests by manually loading the traditional Needleman-Wunsch alignment algorithm and defining a function that gives us a better representation of the alignment. This is not needed in actual research because, as we will see, there are lingpy classes that do this task in a better why, but it is valid while we are understanding how this works. We are going to use two different sets of words: the first is a basic `cat`/`fat` comparison, the second is a more complex and invented data. The sequences in this last set are distinguished by:

- an epenthetic vowel missing in one of the sequences;
- a different place of articulation for the first consonant;
- a different roundness for the first vowel;
- the voiceness of the last consonant.

In [2]:
# manually load the alignment algorithm
from lingpy.align.pairwise import nw_align

# define the two sets for pairwise alignment
seqA1 = 'cat'
seqA2 = 'fat'

seqB1 = 'pirav'
seqB2 = 'itylaf'

# prints a representation of the result of a pairwise alignment
# the alignment functions return a tuple with three values: the alignment for the first sequence,
# the alignment for the second sequence, and the alignment score; the alignment for the first and second sequence
# can be either lists of phonetic units or lists of lists of phonetic units (that is, list of groups of units)
def print_align(align, new_line=False):
    if type(align[0][0]) is str:
        print('seq a: ', ' '.join(align[0]))
        print('seq b: ', ' '.join(align[1]))
    if type(align[0][0]) is list:
        print('seq a: ', '\t\t'.join([' '.join(e) for e in align[0]]))
        print('seq b: ', '\t\t'.join([' '.join(e) for e in align[1]]))
        
    print('score: ', align[2])
    
    # print a new line, separating different alignments, if requested
    if new_line:
        print()
    
# shows the nw alignment for the first set
print_align(nw_align(seqA1, seqA2), new_line=True)

# shows the nw alignment for the second set
print_align(nw_align(seqB1, seqB2))

seq a:  c a t
seq b:  f a t
score:  1.0

seq a:  p i r - - a v
seq b:  - i t y l a f
score:  -3.0


We can see the output of an alignment algorithm: each unit in a sequence is mapped to either a corresponding unit in the other sequence (which may or may not be identical), or to a gap represented by a dash. The algorithm performed as expected for the "cat"/"fat" set, but its alignment in the test case, while not impossible, is not what was expected.

There are many ways to tweak the workings of the alignment algorithms, including a simpler one such as Needleman-Wunsch. One important value to set is the gap penalty, which defaults to -1. Let's see how the algorithm performs when a larger gap is specified.

In [3]:
# shows the nw alignment for the second set, with a larger gap penalty
print_align(nw_align(seqB1, seqB2, gap=-5))

seq a:  p i r - a v
seq b:  i t y l a f
score:  -8.0


The algorithm perfomed even worse, with an unlikely matching of consonants and vowels. Let's see how it performs with a smaller gap.

In [4]:
# shows the nw alignment for the second set, with a smaller gap penalty
print_align(nw_align(seqB1, seqB2, gap=-0.1))

seq a:  p i r - - - a v -
seq b:  - i - t y l a - f
score:  1.2999999999999998


When there is almost no penalty for a gap, the algorithm will likely only match identical units, even when they don't necessarly correspond (remember that, in our test, the `/i/` if "pirav" should match the `/y/` in "itylaf", and not the epenthetic vowel).

There is a function better suited for these pairwise tests, which allows us not only to select between different alignment modes (as detailed in LIST2012a, using "global" as the default mode), but also to specify different penalty scores for gap opening (`gop`, which defaults to -1) and for gap extension scale (`scale`, which defaults to 0.5). Let's experiment with both our sets with the four different alignment modes, using the default values for gap opening and for gap extension scale.

In [5]:
# manually load the alignment algorithm
from lingpy.align.pairwise import pw_align

# shows the pw alignment for the second set, with the (default) global mode
print_align(pw_align(seqB1, seqB2), new_line=True)

# shows the pw alignment for the second set, with the local mode
print_align(pw_align(seqB1, seqB2, mode="local"), new_line=True)

# shows the pw alignment for the second set, with the dialign mode
print_align(pw_align(seqB1, seqB2, mode="dialign"), new_line=True)

# shows the pw alignment for the second set, with the overlap mode
print_align(pw_align(seqB1, seqB2, mode="overlap"), new_line=True)

seq a:  p i - - r a v
seq b:  - i t y l a f
score:  -2.0

seq a:  p i r		a		v
seq b:  i t y l		a		f
score:  1.0

seq a:  p i r a v - - - - -
seq b:  - i - - - t y l a f
score:  1.0

seq a:  p i r a v - - - - - -
seq b:  - - - - - i t y l a f
score:  0.0



Some alignments, such as in "local" above, return groups of phonetic units and might be more adequade. Do not be fooled by the supposedly useless results of "dialign" and "overlap" modes here: they are performing poorly not only because of gap opening and extension penalties, but particularly due to the fact that we are not using sound classes and neither providing the algorithm with a scorer. This is intentional, as you can probably understand, even without a true description, what is favored by each mode when aligning sequences.

It is time to understand how the sound models used by lingpy work, in order to proceed with pairwise and multiple alignments. Before that, it is valid to know that lingpy offers a set of helper functions to deal with phonetic representations and prosodic sequences. The two most important are `sampa2uni()`, which converts from an X-SAMPA to an Unicode representation of IPA, and `ipa2tokens()`, which tokenizes IPA-encoded strings while taking care of details such diacritics.


In [6]:
ipa = lingpy.sampa2uni('tsOyg@')
tokens = lingpy.ipa2tokens(ipa)

print(ipa)
print(tokens)

ʦɔyɡə
['ʦ', 'ɔy', 'ɡ', 'ə']


## Sound classes

Sound classes go back to an approach Dolgopolsky1964. The basic idea behind sound classes is to reduce the IPA space of possible phonetic segments in order to guarantee both the comparability of sounds between languages, but also to give some assessment regarding the probability that sounds belonging to the same class occur in correspondence relations in genetically related languages. More recently, sound classes have been applied in a couple of approaches, including phonetic alignment (see List2012a), and automatic cognate detection (see Turchin2012, List2012b).

Besides `_color`, a model which is used for the coloring of sound-tokens when creating html-output, and `art`, a model which is used for the calculation of sonority profiles and prosodic strings (see List2012), lingpy includes three different sound models that can be used as a starting point for your research:

- `sca` - the SCA sound-class model (see List2012a);
- `dolgo` - the DOLGO sound-class model (see: Dolgopolsky1986);
- `asjp` - the ASJP sound-class model (see Brown2008 and Brown2011).

We will experiment mostly with `sca`, which should be your default. Please remember that it is also possible to specify a user-defined model, which you might need when you advance with your research.

Models are loaded from lingpy resources with the `lingpy.rc()` function and have two essential attributes: a `converter`, used to map IPA-tokens to sound classes, and a `scorer`, used to map two sound classes to a given score used in the alignment (in a simplified and somewhat wrong description, it indicates the tendency for a given sound class to correspond to the other).

Here we show how to load the standard models and how different they are in their conversion of sounds to sound classes (`converter`) and looking at some example of scores for comparing sound classes. You can see that lingpy tends to favor vowels in its alignment (or, more precisely, classes with a tendency to high prosody score), for reasons discussed in List2014.

In [7]:
# loads the standard models from lingpy resources
asjp = lingpy.rc('asjp')
sca = lingpy.rc('sca')
dolgo = lingpy.rc('dolgo')

# shows the different classes to which some sounds are mapped in each model, and the
# score of such class in the model when compared with the class for the
# sound /p/, for the sound /l/, and with class '_' (i.e., a gap)
for sound in ['a', 'b', 's', 'n']:
    for model in [asjp, sca, dolgo]:
        print("({0}) /{1}/ > class '{2}' / {3}, {4}, {5}".format(model.name,
            sound,
            model.converter[sound],
            model.scorer[model.converter[sound], model.converter['p']],
            model.scorer[model.converter[sound], model.converter['l']],
            model.scorer[model.converter[sound], '_']))
    print() # separate with newline

(asjp) /a/ > class 'a' / -8.31, -8.31, -21.88
(sca) /a/ > class 'A' / -10.0, -10.0, -10.0
(dolgo) /a/ > class 'V' / -10.0, -10.0, -20.0

(asjp) /b/ > class 'b' / 4.38, 0.44, -21.88
(sca) /b/ > class 'P' / 10.0, 0.0, -10.0
(dolgo) /b/ > class 'P' / 10.0, 0.0, -20.0

(asjp) /s/ > class 's' / 0.0, 0.44, -21.88
(sca) /s/ > class 'S' / 0.0, 0.0, -10.0
(dolgo) /s/ > class 'S' / 0.0, 0.0, -20.0

(asjp) /n/ > class 'n' / 0.0, 1.31, -21.88
(sca) /n/ > class 'N' / 0.0, 0.0, -10.0
(dolgo) /n/ > class 'N' / 0.0, 0.0, -20.0



### Pairwise alignment with sound classes

We can now experiment with the alignments from the previous section by mapping the sounds of our words to their sound classes and running the alignment algorithms a second time. Please note that this is only using the `converter` from the sound model, but not the scorer: in computational terms, it is still assuming that each sound class is equally distinct from all other sound classes (in other words, the score is always the same). 

We use the `lingpy.ipa2tokens()` function to tokenize our strings and the `lingpy.tokens2class()` function to convert each token to its class in the `sca` model.

In [8]:
# convert our sequences to lists of sound classes in the SCA model
seqA1_sc = lingpy.tokens2class(lingpy.ipa2tokens(seqA1), 'sca')
seqA2_sc = lingpy.tokens2class(lingpy.ipa2tokens(seqA2), 'sca')
seqB1_sc = lingpy.tokens2class(lingpy.ipa2tokens(seqB1), 'sca')
seqB2_sc = lingpy.tokens2class(lingpy.ipa2tokens(seqB2), 'sca')

# show the results of conversions
print('{0} -> {1}'.format(seqA1, seqA1_sc))
print('{0} -> {1}'.format(seqA2, seqA2_sc))
print('{0} -> {1}'.format(seqB1, seqB1_sc))
print('{0} -> {1}'.format(seqB2, seqB2_sc))

# shows the pw alignment for the second set (with sound classes), with the (default) global mode
print_align(pw_align(seqB1_sc, seqB2_sc), new_line=True)

# shows the pw alignment for the second set (with sound classes), with the local mode
print_align(pw_align(seqB1_sc, seqB2_sc, mode="local"), new_line=True)

# shows the pw alignment for the second set (with sound classes), with the dialign mode
print_align(pw_align(seqB1_sc, seqB2_sc, mode="dialign"), new_line=True)

# shows the pw alignment for the second set (with sound classes), with the overlap mode
print_align(pw_align(seqB1_sc, seqB2_sc, mode="overlap"), new_line=True)

cat -> ['C', 'A', 'T']
fat -> ['B', 'A', 'T']
pirav -> ['P', 'I', 'R', 'A', 'B']
itylaf -> ['I', 'T', 'Y', 'L', 'A', 'B']
seq a:  P I - - R A B
seq b:  - I T Y L A B
score:  0.0

seq a:  P I R		A B		
seq b:  I T Y L		A B		
score:  2.0

seq a:  P I R A B - - - - -
seq b:  - I - - - T Y L A B
score:  1.0

seq a:  P I - - R A B
seq b:  - I T Y L A B
score:  0.5



While there are still some problems, we can immediately see that using soundclasses has led to some improvements in the alignment. It should also be possible to understand how a scorer which considers the different distances among sound classes should improve even further the results in some cases, for example with diagonal mode alignment.

LingPy offers a class to automatically apply a model when performing pairwise alignment, similar to what we did above. It also allows to set some properties for the alignment (such as `gop` for the gap opening penalty and `factor` for the factor by which matches in identical prosodic position are increased) and allows to print the results without our ad-hoc `print_align()` method.

Here we test the results of the alignment with different methods, properties, and modes. You are encouraged to explore the results, to tweak it, and to perform tests in words of your choice (remember that `lingpy.sampa2uni()` can help you with IPA input, if needed). Pay attention to the reported alignment score, particularly when the results are apparently the same, and to alignment methods that exclude some sounds from the final results.

In [9]:
from lingpy.align.pairwise import Pairwise

pw_aligner = Pairwise(seqB1, seqB2)

print("Standard alignment with SCA:")
pw_aligner.align(model='sca', pprint=True)

print("\nStandard alignment with ASJP:")
pw_aligner.align(model='asjp', pprint=True)

print("\nStandard alignment with SCA, high gap opening penalty:")
pw_aligner.align(model='sca', gop=-10, pprint=True)

print("\nStandard alignment with SCA, low gap opening penalty:")
pw_aligner.align(model='sca', gop=-0.1, pprint=True)

print("\nStandard alignment with SCA, high gap extension penalty:")
pw_aligner.align(model='sca', scale=2, pprint=True)

print("\nStandard alignment with SCA, low gap extension penalty:")
pw_aligner.align(model='sca', scale=0.1, pprint=True)

print("\nStandard alignment with SCA, high factor of identical prosodic position increase:")
pw_aligner.align(model='sca', factor=1, pprint=True)

print("\nStandard alignment with SCA, low factor of identical prosodic position increase:")
pw_aligner.align(model='sca', factor=0.01, pprint=True)

print("\nStandard alignment with SCA, local alignment mode:")
pw_aligner.align(model='sca', mode='local', pprint=True)

print("\nStandard alignment with SCA, diagonal alignment mode:")
pw_aligner.align(model='sca', mode='dialign', pprint=True)

print("\nStandard alignment with SCA, overlap alignment mode:")
pw_aligner.align(model='sca', mode='overlap', pprint=True)

Standard alignment with SCA:
p	i	-	-	r	a	v
-	i	t	y	l	a	f
27.6

Standard alignment with ASJP:
p	i	-	-	r	a	v
-	i	t	y	l	a	f
35.647

Standard alignment with SCA, high gap opening penalty:
-	p	i	r	a	v
i	t	y	l	a	f
13.7

Standard alignment with SCA, low gap opening penalty:
p	i	-	-	r	a	v
-	i	t	y	l	a	f
30.84

Standard alignment with SCA, high gap extension penalty:
-	p	i	r	a	v
i	t	y	l	a	f
22.7

Standard alignment with SCA, low gap extension penalty:
p	i	-	-	r	a	v
-	i	t	y	l	a	f
29.44

Standard alignment with SCA, high factor of identical prosodic position increase:
p	i	-	-	r	a	v
-	i	t	y	l	a	f
44.4

Standard alignment with SCA, low factor of identical prosodic position increase:
-	p	i	r	a	v
i	t	y	l	a	f
21.689999999999998

Standard alignment with SCA, local alignment mode:
p	i	r	a	v
t	y	l	a	f
29.3

Standard alignment with SCA, diagonal alignment mode:
p	i	-	-	r	a	v
-	i	t	y	l	a	f
29.3

Standard alignment with SCA, overlap alignment mode:
-	p	i	r	a	v
i	t	y	l	a	f
29.3


## Multiple alignment

The next obvious step from pairwise alignment is multiple alignment, which is what you are going to deal with when researching language families and candidates for cognancy. As in the case with pairwise alignemnt, lingpy offers a class that allows you to quickly check the results for the alignment of sets of multiple words.

Let's start by exploring how lingpy aligns names related to a famous character of the Harry Potter lore. We are intentionally including two names ("Tom Riddle" and "Harry") that clearly are not cognates, so that we can use the same set later when exploring cognancy. We are aware that the strings here presented are not valid IPA transcriptions of the pronounciation of the names in most languages and that there are inconsistencies: it is good enough for our demonstration, and using plain ASCII characters simplies the analysis.

In [10]:
hp_names = ['woldemort','waldemar','wladimir','vladymyr', 'tomriddle', 'harry']
msa = lingpy.Multiple(hp_names)
msa.prog_align()
print(msa)

w	o	l	-	d	e	m	o	r	-	t	-	-
w	a	l	-	d	e	m	a	r	-	-	-	-
w	-	l	a	d	i	m	i	r	-	-	-	-
v	-	l	a	d	y	m	y	r	-	-	-	-
-	-	-	-	t	o	m	-	r	i	dd	l	e
-	-	-	-	h	a	-	-	rr	-	-	-	y


The results are very good if we consider that only the first four words are candidates for cognancy. Still, the algorithm was able to align in the most acceptable way the two outliers: for example, the double /d/ of "Tom Riddle" was aligned with the voiceless /t/ of "Woldemort".

One aspect that might draw your attention is the alignment of the first sounds in the cognates: the fact that you either have a vowel before or after the /l/ would be takes in historical linguistics as a strong indicator for metathesis. This sound change is, in fact, one of the hard parts for both automatic and manual sound alignment and will not be covered here (the impact of this difficulty in actual studies is not as strong as it may seem at first sight). If you are interested, you can study the discussion on the topic in LIST2014, and be advised that lingpy offers `swap_check()` a method to check for the possibility of swapped sited in the alignment, as shown below using a restricted version of our toy dataset.

In [11]:
# restricted dataset with high potential case for site swapping
swap_aligner = lingpy.Multiple(["woldemort", "waldemar", "wladimir"])
swap_aligner.prog_align()
swap_aligner.swap_check()

# prints the analysis of `swap_check()` in the restricted and in the
# full dataset
print('swap_check() in restricted: ', swap_aligner.swap_check())
print('swap_check() in full: ', msa.swap_check())

swap_check() in restricted:  True
swap_check() in full:  False


## Basic formats

After understanding the basic properties of pairwise and multiple alignment, the building block of lingpy, we can start looking at real data and conducting real analysis.

The basic input format read by LingPy is a comma or tab delimited text file in which the first line (the header) indicates the values of the columns and all words are listed in the following rows. The format is very flexible. No specific order of columns or rows is required. Any additional data can be specified by the user, as long as it is in a separate column. Each row represents a word that has to be characterized by a minimum of four values that are given in separate columns: (1) ID, an integer that is used to uniquely identify the word during calculations, (2) CONCEPT, a gloss which indicates the meaning of the word and which is used to align the words semantically, (3) WORD, the orthographic representation of the word, and (4) DOCULECT, the name of the language (or dialect) in which the word occurs. Basic output formats are essentially the same, the difference being that the results of calculations are added as separate columns. [LIST2013]

### Wordlist

For the Wordlist class (and also for all classes that inherit from it, such as LexStat, PhyBo, Alignments), a simple csv-format is used. This format is a simple comma or tab delimited text file in which the header specifies all entry types in a given dataset. This format can be further extended by adding key-value pairs in the lines before the header, such as, for example, information regarding the author, the data, or general notes.

In the example below, we have 12 different entries ("words") that express four different semantic groups or "concepts" ("hand", "leg", "Woldemort", and "Harry") in 4 different languages (German, English, Russian, and Ukrainian). The "counterpart" is a representation of the entry, and in most cases it will be its written form in a script used by its language, and IPA, as expected, is a phonemic representation of the pronunciation (as for why it is better to use a phonemic and not a phonetic representation, please refer to the essential bibliography). The cognate ID groups entries by a shared common ancestor, with unique identification numbers for unique ancestors of a concept. So, for example, German `"Hand"` and English `"hand"` share a cognate ID due to both descending from Proto-Germanic `*handuz`, and Russian `"рука"` and Ukrainian `"рука"` share a different cognate ID due to its descending from Proto-Slavic `*rǫka`; at the same time, all entries for concept "Woldemort" share a cognate ID as they all descend (in some cases due to borrowing). It is important to know that an entry might be the only representative of its cognate set, and in fact this is common (see the case of entry number 5, German `"Bein"`). 

```
ID   CONCEPT     COUNTERPART   IPA         DOCULECT     COGID
1    hand        Hand          hant        German       1
2    hand        hand          hænd        English      1
3    hand        рука          ruka        Russian      2
4    hand        рука          ruka        Ukrainian    2
5    leg         Bein          bain        German       3
6    leg         leg           lɛg         English      4
7    leg         нога          noga        Russian      5
8    leg         нога          noha        Ukrainian    5
9    Woldemort   Waldemar      valdemar    German       6
10   Woldemort   Woldemort     wɔldemɔrt   English      6
11   Woldemort   Владимир      vladimir    Russian      6
12   Woldemort   Володимир     volodimir   Ukrainian    6
9    Harry       Harald        haralt      German       7
10   Harry       Harry         hæri        English      8
11   Harry       Гарри         gari        Russian      8
12   Harry       Гаррi         hari        Ukrainian    8
```

This are exactly the contents of file `toy_wordlist.wl` distibuted along with this tutorial and loaded, in the code below, with function `get_wordlist()`. Some queries demonstrate that the list was loaded correctly; please remember that, while it obviously does not try to replace a database system, the Wordlist class offers many helper function to deal with the data, such as `add_entries()`, `renumber()`, and `output()`.

In [12]:
from lingpy.basic.wordlist import get_wordlist

wl = get_wordlist(test_data('tutorial.qlc'), delimiter='\t')

print(wl.get_list(language="German"))
print(wl.get_dict(language="Russian"))

[9, 5, 1, 13]
defaultdict(<class 'list'>, {'Woldemort': [11], 'leg': [7], 'hand': [3], 'Harry': [15]})


### Pairwise phonetic alignments

The input format for text files containing unaligned sequence pairs is called PSQ-format. Files in this format should have the extension psq. The first line of a PSQ-file contains information regarding the dataset. The sequence pairs are given in triplets, with a sequence identifier in the first line of a triplet (containing the meaning, or orthographical information) and the two sequences in the second and third line, whereas the first column of each sequence line contains the name of the taxon and the second column the sequence in IPA format; columns are separated by tabs, sounds by spaces. All triplets are divided by one empty line. As an example, consider the file `harry_potter.psq`:

```
Harry Potter Testset
Woldemort in German and Russian
German      w a l d e m a r
Russian     v l a d i m i r
 
Woldemort in English and Russian
English     w o l d e m o r t
Russian     v l a d i m i r
 
Woldemort in English and German
English     w o l d e m o r t
German      w a l d e m a r
```

In [13]:
from lingpy.align.sca import PSA

psa = PSA(infile=test_data('harry_potter.psq'))

print(psa.pairs)

[('w a l d e m a r', 'v l a d i m i r'), ('w o l d e m o r t', 'v l a d i m i r'), ('w o l d e m o r t', 'w a l d e m a r')]


The output counterpart of the PSQ-format is the PSA-format. It is a specific format for text files containing already aligned sequence pairs. Files in this format should have the extension psa. The first line of a PSA-file contains information regarding the dataset. The sequence pairs are given in triplets, with a sequence identifier in the first line of a triplet (containing the meaning, or orthographical information) and the aligned sequences in the second and third line, whith the name of the taxon in the first column and all aligned segments in the following columns, separated by tabstops. All triplets are divided by one empty line. As an example, consider the file `harry_potter.psa`:

```
Harry Potter Testset
Woldemort in German and Russian
German.     w     a     l     -     d     e     m     a     r
Russian     v     -     l     a     d     i     m     i     r
 
Woldemort in English and Russian
English     w     o     l     -     d     e     m     o     r     t
Russian     v     -     l     a     d     i     m     i     r     -

Woldemort in English and German
English     w     o     l     d     e     m     o     r     t
German.     w     a     l     d     e     m     a     r     -
```

In [14]:
pass

### Multiple Alignments (MSQ and MSA)

A specific format for text files containing multiple unaligned sequences is the MSQ-format. Files in this format should have the extension msq. The first line of an msq-file contains information regarding the dataset. The second line contains information regarding the sequence (meaning, identifier), and the following lines contain the name of the taxa in the first column and the sequences in IPA format in the second column, separated by a tabstop. As an example, consider the file harry_potter.msq:
    
```
1 Harry Potter Testset
2 Woldemort (in different languages)
3 English     v o l d e m o r t
4 German      w a l d e m a r
5 Russian     v l a d i m i r
```

In [15]:
pass

The msa-format is a specific format for text files containing already aligned sequence pairs. Files in this format should have the extension msa. The first line of a MSA-file contains information regarding the dataset. The second line contains information regarding the sequence (its meaning, the protoform corresponding to the cognate set, etc.). The aligned sequences are given in the following lines, whereas the taxa are given in the first column and the aligned segments in the following columns. Additionally, there may be a specific line indicating the presence of swaps and a specific line indicating highly consistent sites (local peaks) in the MSA. The line for swaps starts with the headword SWAPS whereas a plus character (+) marks the beginning of a swapped region, the dash character (-) its center and another plus character the end. All sites which are not affected by swaps contain a dot. The line for local peaks starts with the headword LOCAL. All sites which are highly consistent are marked with an asterisk (*), all other sites are marked with a dot (.). As an example, consider the file harry_potter.msa:

```
Harry Potter Testset
Woldemort (in different languages)
English     v     o     l     -     d     e     m     o     r     t
German.     w     a     l     -     d     e     m     a     r     -
Russian     v     -     l     a     d     i     m     i     r     -
SWAPS..     .     +     -     +     .     .     .     .     .     .
LOCAL.      *     *     *     .     *     *     *     *     *     .
```

### search automatically for cognates across multiple languages

LingPy includes LexStat, a powerful method for automatic detection of cognates. In fact, while relying on the methods for pairwise and multiple comparison describe above, LexStat draws from traditional practices in historical linguistics. In general, the procedures for cognate detection in the traditional approach could be described in the following way:

- Compile an initial list of putative cognate sets
- Extract an initial list of putative sets of sound correspondences from the initial cognate list
- Refine the cognate list and the correspondences list by
    - adding and deleting cognate sets from the cognate list, depending on whether they are consistent with the correspondence list or not, and
    - adding and deleting correspondence sets from the correspondence list, depending on whether they are consistent with the cognate list or not
- Finish when the results are satisfying enough.

The cognate detection with LexStat uses a theoretically similar model in four steps:

- Sequence conversion, in which sequences are converted to sound classes and prosodic profiles (all sequences are internally represented both as sound classes and by prosodic strings, which indicate the prosodic environment of each phonetic segment: initial, ascending, maximum, descending, final)
- Scoring-Scheme creation, in which language-specific scoring schemes are determined using a permutation method (global and pairwise alignment analyses of all sequence pairs occuring in the same semantic slot whose distance is beyong a certain threshold are stored, the wordlists are shuffled repeatedly and the average results are stored, so that log-odd scores from the distributions can be calculated)
- Distance calculation, in which pairwise distances between sequences are calculated based on the language-specific scoring-scheme
- Sequence clustering, in which sequences are clustered into cognate sets whose average distance is beyond a certain threshold

Here, we are going to load a complex dataset with 200 concepts in English, German, French, Albanian, Turkish, Hawaiian, and Navajo (a Swadesh wordlist), and apply LexStat to search for cognates. The dataset is part of the standard lingpy distribution, and it includes a cognate identification column with can be used a gold standard for testing the perfomance of our automatic cognate identification.

Remember that it is a good practice to set `check=True` when loading a dataset, so that you can catch errors early on, before they propagate.

In [16]:
# load the data and check its contents
lex = lingpy.LexStat(test_data('KSL.qlc'), check=True)

print('number of languages: ', lex.width)
print('number of concepts: ', lex.height)
print('number of entries: ', len(lex))

# show coverage (a dataset might not have full coverage for some or all languages)
print(lex.coverage())

2017-02-11 18:11:32,481 [INFO] No obvious errors found in the data.


number of languages:  7
number of concepts:  200
number of entries:  1400
{'Turkish': 200, 'English': 200, 'Hawaiian': 200, 'Albanian': 200, 'German': 200, 'Navajo': 200, 'French': 200}


An essential method of LexStat is `get_scorer()`, which creates a scoring function based on sound correspondences. A number of different properties can be set to control the correspondence calculation, including:

- `ratio`, which defines the ratio between derived and original score for sound matches
- `vscale`, which defines a scaling factor for vowels, in order to decrease their score in the calculations
- `threshold`, which defines the threshold used to select those words that are compared in order to derive the attested distribution
- `unattested`, which allows to set the value for a pair of sounds which is expected by the alignment algorithm but which is not attesed in the data (whose score would otherwise be -infinity)

Running `get_scorer()` should take a little time, as it calculates both the attested and the random alignments for all language pairs.

In [17]:
lex.get_scorer()

CORRESPONDENCE CALCULATION:   0%|          | 0/24.5 [00:00<?, ?it/s]2017-02-11 18:11:33,254 [INFO] Calculating alignments for pair Albanian / Albanian.
CORRESPONDENCE CALCULATION:   8%|▊         | 2/24.5 [00:00<00:01, 19.72it/s]2017-02-11 18:11:33,363 [INFO] Calculating alignments for pair Albanian / English.
CORRESPONDENCE CALCULATION:  12%|█▏        | 3/24.5 [00:00<00:01, 14.95it/s]2017-02-11 18:11:33,462 [INFO] Calculating alignments for pair Albanian / French.
CORRESPONDENCE CALCULATION:  16%|█▋        | 4/24.5 [00:00<00:01, 12.76it/s]2017-02-11 18:11:33,568 [INFO] Calculating alignments for pair Albanian / German.
CORRESPONDENCE CALCULATION:  20%|██        | 5/24.5 [00:00<00:01, 11.56it/s]2017-02-11 18:11:33,673 [INFO] Calculating alignments for pair Albanian / Hawaiian.
CORRESPONDENCE CALCULATION:  24%|██▍       | 6/24.5 [00:00<00:01, 10.52it/s]2017-02-11 18:11:33,788 [INFO] Calculating alignments for pair Albanian / Navajo.
CORRESPONDENCE CALCULATION:  29%|██▊       | 7/24.5 [00

Once the calculation of the scoring function for our data is completed, we can run the methods for the analyses we desire, such as the one for flat clustering the words in the dataset to cognate sets (ie., to estimate the COGID).

Given that we have a column with a golden standard for the COCID, we can analyze how well the automatic cognate detection is working with statistical methods such as the B-Cubed scores (Bagga1999). These rank between 0 and 1, with one being good and 0 being bad, and come in three flavors of precision (amount of false positives), recall (amount of false negatives) and F-Score (combined score). We compute them by passing the wordlist object, the gold standard (stored in column `"cogid"` in our file), and our computed cognate sets (stored in the column we specify using the `"ref"` keyword). Let's experiment with different methods and thresholds (please note that this computation can take some time):

In [18]:
from lingpy.evaluate.acd import bcubes, diff

# do the clustering
lex.cluster(method="sca", threshold=0.6, ref="cognates_sca")
lex.cluster(method="lexstat", threshold=0.6, ref="cognates_lexstat")
lex.cluster(method="edit-dist", threshold=0.6, ref="cognates_editdist")
lex.cluster(method="turchin", threshold=0.6, ref="cognates_turchin")

# evaluate
print("SCA, threshold=0.6 (precision/recall/f-score): ", bcubes(lex, "cogid", "cognates_sca", pprint=False))
print("LexStat, threshold=0.6 (precision/recall/f-score): ", bcubes(lex, "cogid", "cognates_lexstat", pprint=False))
print("EditDist, threshold=0.6 (precision/recall/f-score): ", bcubes(lex, "cogid", "cognates_editdist", pprint=False))
print("Turchin, threshold=0.6 (precision/recall/f-score): ", bcubes(lex, "cogid", "cognates_turchin", pprint=False))

SEQUENCE CLUSTERING:   0%|          | 0/200 [00:00<?, ?it/s]2017-02-11 18:11:51,852 [INFO] Analyzing words for concept <I>.
2017-02-11 18:11:51,861 [INFO] Analyzing words for concept <all>.
2017-02-11 18:11:51,872 [INFO] Analyzing words for concept <and>.
2017-02-11 18:11:51,884 [INFO] Analyzing words for concept <animal>.
2017-02-11 18:11:51,900 [INFO] Analyzing words for concept <ashes>.
2017-02-11 18:11:51,909 [INFO] Analyzing words for concept <at>.
2017-02-11 18:11:51,920 [INFO] Analyzing words for concept <back>.
2017-02-11 18:11:51,931 [INFO] Analyzing words for concept <bad>.
2017-02-11 18:11:51,944 [INFO] Analyzing words for concept <bark>.
SEQUENCE CLUSTERING:   4%|▍         | 9/200 [00:00<00:02, 85.80it/s]2017-02-11 18:11:51,961 [INFO] Analyzing words for concept <because>.
2017-02-11 18:11:51,973 [INFO] Analyzing words for concept <belly>.
2017-02-11 18:11:51,984 [INFO] Analyzing words for concept <big>.
2017-02-11 18:11:51,995 [INFO] Analyzing words for concept <bird>.
201

SCA, threshold=0.6 (precision/recall/f-score):  (0.6856904761904768, 0.9672619047619042, 0.8024941114092562)
LexStat, threshold=0.6 (precision/recall/f-score):  (0.9285714285714286, 0.9472619047619046, 0.9378235523440829)
EditDist, threshold=0.6 (precision/recall/f-score):  (0.9540476190476191, 0.9114285714285715, 0.9322512535326831)
Turchin, threshold=0.6 (precision/recall/f-score):  (0.9310714285714288, 0.9277380952380955, 0.9294017731340009)


In this particular dataset and with the parameters we used, the get the best f-score is probably the one of the LexStat method (it might differ in your computer, as the wordlist is randomly shuffled). This score is rather high, but we should keep in mind that the dataset is rather small. Just for fun, let's try the same method with higher and lower thresholds, and see how it performs.

In [19]:
lex.cluster(method="lexstat", threshold=0.3, ref="cognates_lexstat03")
lex.cluster(method="lexstat", threshold=0.9, ref="cognates_lexstat09")

print("LexStat, threshold=0.3 (precision/recall/f-score): ", bcubes(lex, "cogid", "cognates_lexstat03", pprint=False))
print("LexStat, threshold=0.9 (precision/recall/f-score): ", bcubes(lex, "cogid", "cognates_lexstat09", pprint=False))

SEQUENCE CLUSTERING:   0%|          | 0/200 [00:00<?, ?it/s]2017-02-11 18:12:01,213 [INFO] Analyzing words for concept <I>.
2017-02-11 18:12:01,224 [INFO] Analyzing words for concept <all>.
2017-02-11 18:12:01,239 [INFO] Analyzing words for concept <and>.
2017-02-11 18:12:01,250 [INFO] Analyzing words for concept <animal>.
2017-02-11 18:12:01,273 [INFO] Analyzing words for concept <ashes>.
2017-02-11 18:12:01,288 [INFO] Analyzing words for concept <at>.
2017-02-11 18:12:01,298 [INFO] Analyzing words for concept <back>.
2017-02-11 18:12:01,313 [INFO] Analyzing words for concept <bad>.
SEQUENCE CLUSTERING:   4%|▍         | 8/200 [00:00<00:02, 68.80it/s]2017-02-11 18:12:01,332 [INFO] Analyzing words for concept <bark>.
2017-02-11 18:12:01,352 [INFO] Analyzing words for concept <because>.
2017-02-11 18:12:01,368 [INFO] Analyzing words for concept <belly>.
2017-02-11 18:12:01,384 [INFO] Analyzing words for concept <big>.
2017-02-11 18:12:01,400 [INFO] Analyzing words for concept <bird>.
201

LexStat, threshold=0.3 (precision/recall/f-score):  (0.9957142857142857, 0.8897619047619045, 0.9397611531037467)
LexStat, threshold=0.9 (precision/recall/f-score):  (0.5130170068027222, 0.9804761904761902, 0.6735898916659002)


With this particular dataset, you probably got a higher score with a lower threshold. There is no golden rule here, as each method will require different parameters and a higher score may not necessarily be what you are looking for (for example, if the golden standard is biased and you are using lingpy precisely to understand how). Once more, lingpy is a tool for computer *assisted* historical linguistics.

### Alignments and Philogeny

Now that we know that our data has been properly analyzed with good cognate scores, lets align it, using the Alignments class. We can directly initialize it from the LexStat object, but we need to pass the “cognates” column as “ref” (this tells LingPy, where to search for cognate sets which are then multiply aligned), and then, we call the function align, using the defaults for convenience:

In [20]:
alm = lingpy.Alignments(lex, ref='cognates_lexstat')
alm.align()

If you want to see the results of this analysis, you need to write them to file. But there, it is also difficult to see the alignments, since they are in a TSV-file in just another column, called “alignment” as a default. Another possibility is to write data to HTML format instead. This means you can’t re-import the data into LingPy, but you can inspect the results. This will create the file KSL.html which you can inspect by loading it in your webbrowser:

In [21]:
alm.output('html', filename="align_KSL")

2017-02-11 18:12:09,272 [INFO] Data has been written to file </tmp/tmp7lg6no9l.alm>.
2017-02-11 18:12:09,405 [INFO] Data has been written to file <align_KSL.html>.


Finally, let’s make a tree of the data. This is very straightforward by passing the “cognates” column as a reference to the calculate function. Printing of the resulting “tree” which is created as an attribute of the LexStat object, is possible with help of LingPy’s Tree class: Well, given that there are unrelated languages in our sample, we should be careful with any interpretations here, but let’s at least be glad that the algorithm did cluster the Indo-European languages all together.

In [22]:
lex.calculate('tree', ref='cognates_lexstat')
print(lex.tree.asciiArt())

2017-02-11 18:12:09,565 [INFO] Successfully calculated tree.


          /-Hawaiian
-root----|
         |          /-Navajo
          \edge.4--|
                   |          /-Turkish
                    \edge.3--|
                             |          /-Albanian
                              \edge.2--|
                                       |          /-French
                                        \edge.1--|
                                                 |          /-English
                                                  \edge.0--|
                                                            \-German


### calculate lexicostatistic distances between languages

In [23]:
pass

### reconstruct language phylogenies using basic cluster algorithm

In [24]:
from lingpy.algorithm import squareform
languages = ['Norwegian','Swedish','Icelandic','Dutch','English']
distances = squareform([0.5,0.67,0.8,0.2,0.4,0.7,0.6,0.8,0.8,0.3])
tree = lingpy.neighbor(distances,languages)
print(tree)


(((Norwegian:0.18,(Swedish:0.12,Icelandic:0.28):0.20):0.17,English:-0.01):0.16,Dutch:0.16);


Print the tree in ASCII-Art to the screen (using PyCogent’s tree class)

In [25]:
n_tree = lingpy.Tree(tree)
print(n_tree.asciiArt())

                              /-Norwegian
                    /edge.1--|
                   |         |          /-Swedish
          /edge.2--|          \edge.0--|
         |         |                    \-Icelandic
-root----|         |
         |          \-English
         |
          \-Dutch


# What else?

- generate random sequences
- concept comparison
- Compute the Infomap clustering analysis of the data (requires...)
- affinity propagaion (requires sklearn to run the analysis)

For advanced users, you can manually inspect all the builtin parameters with the code below. Please note that this is only intended for debugging and studying, as the standard way to communicate with `rcParams` (which is just a Python dictionary) is by using the `lingpy.rc()` function as we did above.

For multiple alignment, we are here using progressive alignment analysis (method `prog_align()`) in contrast to library-based progressive alignment analysis (method `lib_align()`); the results shouldn't differ for most test cases and you can study their difference later.

In [26]:
import lingpy.settings
lingpy.settings.rcParams

ipa = lingpy.sampa2uni('tsOyg@') # /ʦɔyɡə/ in X-SAMPA
tokens = lingpy.ipa2tokens(ipa) # Tokenize IPA-encoded strings.
classes = lingpy.tokens2class(tokens, 'sca') # Convert tokenized IPA strings into their respective class strings.
prosody = lingpy.prosodic_string(tokens)
prosodic_weights = lingpy.prosodic_weights(prosody)